# Асинхронное программирование 

## Работа с другими процессами 

In [3]:
import subprocess
p = subprocess.run(['python', '-c', 'open("/tmp/5555", "w").close()'])

In [4]:
p = subprocess.run(
    ['perl', '-E', 'say "out"; warn "err\n"'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

In [5]:
p.stdout, p.stderr

(b'out\n', b'err\n')

In [10]:
from subprocess import Popen

p = Popen(
    ['perl', '-E', 'say "out"; warn "err\n"'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

In [7]:
p.communicate()

(b'out\n', b'err\n')

In [11]:
while True:
    out = p.stdout.readline()
    err = p.stderr.readline()
    if out:
        print(f'out: {out}')
    if err:
        print(f'err: {out}')
    if not out and not err:
        break

out: b'out\n'
err: b'out\n'


In [9]:
p = Popen(
    ['perl', '-E', 'warn "err\n" x 1000000; say "out";'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

while True:
    out = p.stdout.readline()
    err = p.stderr.readline()
    if out:
        print(f'out: {out}')
    if err:
        print(f'err: {out}')
    if not out and not err:
        break

KeyboardInterrupt: 

## Процессы

In [12]:
import os
from multiprocessing import Process
 
def doubler(number):
    """
    Функция умножитель на два
    """
    result = number * 2
    proc = os.getpid()
    print('{0} doubled to {1} by process id: {2}'.format(
        number, result, proc))

numbers = [5, 10, 15, 20, 25, 30, 35, 40, 45, 46, 50, 55, 60, 65, 70]
procs = []

for index, number in enumerate(numbers):
    proc = Process(target=doubler, args=(number,))
    procs.append(proc)
    proc.start()

for proc in procs:
    proc.join()

5 doubled to 10 by process id: 14207
10 doubled to 20 by process id: 14210
20 doubled to 40 by process id: 14214
15 doubled to 30 by process id: 14213
25 doubled to 50 by process id: 14219
30 doubled to 60 by process id: 14222
35 doubled to 70 by process id: 14225
40 doubled to 80 by process id: 14228
45 doubled to 90 by process id: 14231
46 doubled to 92 by process id: 14234
50 doubled to 100 by process id: 14237
55 doubled to 110 by process id: 14240
60 doubled to 120 by process id: 14241
65 doubled to 130 by process id: 14246
70 doubled to 140 by process id: 14248


In [13]:
os.cpu_count()

8

In [14]:
p = Popen(
    ['perl', '-E', 'warn "err\n"; say "out";'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

def log(fh, prefix):
    while True:
        line = fh.readline()
        if line:
            print('{}: {}'.format(prefix, line.decode('utf8')), end='')
        else:
            break

processes = [
    Process(target=log, args=(p.stdout, 'out')),
    Process(target=log, args=(p.stderr, 'err')),
]
for p in processes:
    p.start()
for p in processes:
    p.join()

out: out
err: err


## Треды

In [15]:
import threading
threads_num = 4

p = Popen(
    ['perl', '-E', 'warn "err\n"; say "out";'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

def log(fh, prefix):
    while True:
        line = fh.readline()
        if line:
            print('{}: {}'.format(prefix, line.decode('utf8')), end='')
        else:
            break

threads = [
    threading.Thread(target=log, args=(p.stdout, 'out')),
    threading.Thread(target=log, args=(p.stderr, 'err')),
]
for t in threads:
    t.start()
for t in threads:
    t.join()

err: err
out: out


## asyncio

In [57]:
import asyncio

In [58]:
p = asyncio.create_subprocess_exec(
    'perl', '-E', 'warn "err\n"; say "out"; warn "2\n"',
    stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
)

In [59]:
p

<coroutine object create_subprocess_exec at 0x7f67185bcb48>

In [61]:
async def log(fh, prefix):
    while True:
        line = await fh.readline()
        if line:
            print('{}: {}'.format(prefix, line))
        else:
            break

async def execute(loop):
    p = await asyncio.create_subprocess_exec(
        'perl', '-E', 'warn "err\n"; say "out"; warn "2\n"',
        stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
    )
    
    loop.create_task(log(p.stdout, 'stdout'))
    loop.create_task(log(p.stderr, 'stderr'))
    
    await p.wait()

In [62]:
loop = asyncio.get_event_loop()

execute(loop)

asyncio.ensure_future(execute(loop))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: coroutine 'execute' was never awaited
  This is separate from the ipykernel package so we can avoid doing imports until


<Task pending coro=<execute() running at <ipython-input-61-fba4bf4663a6>:9>>

stdout: b'out\n'
stderr: b'err\n'
stderr: b'2\n'


## aiohttp

In [65]:
import aiohttp
import asyncio

async def fetch(session, url):
    async with session.get(url) as response:
        if response.status == 200:
            return await response.text()  # return yield from
        else:
            return f'ERROR: {response.status}'

async def download_wiki(article):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, 'http://de.wikipedia.org/{}'.format(article))
        return html[:15]

loop = asyncio.get_event_loop()
tasks = asyncio.gather(
    download_wiki('wiki/Évariste_Galois'),
    download_wiki('wiki/Alan_Turing'),
    download_wiki('zzz'),
)

In [66]:
tasks

<_GatheringFuture finished result=['<!DOCTYPE html>', '<!DOCTYPE html>', 'ERROR: 404']>